In [1]:
# For readability: disable warnings from libraries like matplotlib, etc.
import warnings
warnings.filterwarnings('ignore')

import os
# Make sure torch is imported somewhere above this cell:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
import time
from itertools import product, combinations
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import scipy.sparse as sp
import scipy.sparse.linalg as la
from pyDOE import lhs
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogLocator, FuncFormatter
from matplotlib.ticker import FormatStrFormatter
import copy
import pandas as pd

# --- Device Setup ---
print("CUDA available?", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))

# Select the most performant device available (CUDA > MPS > CPU)
device = (
    torch.device('cuda') if torch.cuda.is_available()
    else torch.device('mps') if hasattr(torch.backends, "mps") and torch.backends.mps.is_available()
    else torch.device('cpu')
)
print("Using device:", device)


def u_true_numpy(X, T, K):
    """Vectorised true solution: U = exp(-(pi^2) * T) * sin(pi * X)."""
    D = np.tanh(K)
    return np.exp(- D * (np.pi**2) * T) * np.sin(np.pi * X)

def net_u(x, t, k, model):
    """
    NN input is (x, t, l).
    """
    X = torch.cat([x, t, k], dim=1)  # If x and t are each shape (N, 1), then X becomes (N, 2).
    u = model(X)
    return u

# net_f computes the PDE residual
# If f ≈ 0 at collocation points, the NN satisfies the equation there
def net_f(x, t, k, model):
    x.requires_grad_(True)
    t.requires_grad_(True)
    # k.requires_grad_(True)
    
    u = net_u(x, t, k, model)
    
    u_x = torch.autograd.grad(
        u, x, grad_outputs=torch.ones_like(u),
        create_graph=True, retain_graph=True
    )[0]
    
    u_t = torch.autograd.grad(
        u, t, grad_outputs=torch.ones_like(u),
        create_graph=True
    )[0]
    
    u_xx = torch.autograd.grad(
        u_x, x, grad_outputs=torch.ones_like(u_x),
        create_graph=True
    )[0]
    
    D = torch.tanh(k)       # diffusivity = tanh(k)
    f = u_t - D * u_xx      # u_t = tanh(k) * u_xx
    return f

class XavierInit(nn.Module):
    def __init__(self, size):
        super(XavierInit, self).__init__()
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = torch.sqrt(torch.tensor(2.0 / (in_dim + out_dim)))
        self.weight = nn.Parameter(torch.randn(in_dim, out_dim) * xavier_stddev)
        self.bias = nn.Parameter(torch.zeros(out_dim))

    def forward(self, x):
        return torch.matmul(x, self.weight) + self.bias

def initialize_NN(layers):
    weights = nn.ModuleList()
    num_layers = len(layers)
    for l in range(num_layers - 1):
        layer = XavierInit(size=[layers[l], layers[l + 1]]) # if there was no retutn, how do I get the weight and bias?
        weights.append(layer)
    return weights

class NeuralNet(nn.Module):
    def __init__(self, layers, lb, ub):
        super(NeuralNet, self).__init__()
        self.weights = initialize_NN(layers)
        # make lb/ub move with .to(device)
        self.register_buffer('lb', torch.as_tensor(lb, dtype=torch.float32))     # <<< CHANGED >>>
        self.register_buffer('ub', torch.as_tensor(ub, dtype=torch.float32))     # <<< CHANGED >>>
        # self.register_buffer('k', torch.tensor(k_init, dtype=torch.float32))     # <<< CHANGED >>>


    def forward(self, X):
        X = X.float()                                                            # <<< CHANGED >>>
        lb = self.lb.to(X.device)                                                # <<< CHANGED >>>
        ub = self.ub.to(X.device)                                                # <<< CHANGED >>>
        H = 2.0 * (X - self.lb) / (self.ub - self.lb) - 1.0
        for l in range(len(self.weights) - 1):
            H = torch.tanh(self.weights[l](H.float()))     # Is this already a calculation?
        Y = self.weights[-1](H)
        return Y

def train(nEpoch, X, u, X_f, X_val, model, learning_rate):
    criterion = nn.MSELoss()
    
    # ----- STAGE 1: start with Adam -----
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # when to switch from Adam to L-BFGS
    switch_epoch = 3000.0
    used_lbfgs   = False   
    lbfgs_epochs   = 3000.0           # <-- how many epochs of L-BFGS you want
    lbfgs_start_ep = None             # <-- will store the epoch where we switch

    # use the model's device
    dev = next(model.parameters()).device                                        # <<< CHANGED >>>

    x    = X[:, 0:1]
    t    = X[:, 1:2]
    k    = X[:, 2:3]
    # Collocation points (f points)
    x_f    = X_f[:, 0:1]
    t_f    = X_f[:, 1:2]
    k_f    = X_f[:, 2:3]
    # Validation points
    x_v    = X_val[:, 0:1]
    t_v    = X_val[:, 1:2]
    k_v    = X_val[:, 2:3]

    # True validation solution (analytic)
    u_v_true = u_true_numpy(x_v, t_v, k_v)   # shape (N_val,)

    # ------------- create tensors ON THE SAME DEVICE -------------
    x_tf   = torch.tensor(x,   dtype=torch.float32, device=dev, requires_grad=True)
    t_tf   = torch.tensor(t,   dtype=torch.float32, device=dev, requires_grad=True)
    k_tf   = torch.tensor(k,   dtype=torch.float32, device=dev, requires_grad=True)

    u_tf   = torch.tensor(u,   dtype=torch.float32, device=dev)  # training data (no grad)

    x_f_tf = torch.tensor(x_f, dtype=torch.float32, device=dev, requires_grad=True)
    t_f_tf = torch.tensor(t_f, dtype=torch.float32, device=dev, requires_grad=True)
    k_f_tf = torch.tensor(k_f, dtype=torch.float32, device=dev, requires_grad=True)

    x_v_tf    = torch.tensor(x_v,      dtype=torch.float32, device=dev)
    t_v_tf    = torch.tensor(t_v,      dtype=torch.float32, device=dev)
    k_v_tf    = torch.tensor(k_v,      dtype=torch.float32, device=dev)
    u_true_tf = torch.tensor(u_v_true, dtype=torch.float32, device=dev).reshape(-1, 1)

    mse_v_hist  = []
    loss_values = []
    max_errors  = []   # <<< NEW: to store (epoch, max_abs_error) >>>

    patience  = 10000          # number of validations without improvement
    pat       = 0
    best_v    = float('inf')   # best validation MSE
    best_TL   = float('inf')   # best training loss corresponding to best_v
    best_max_err = float('inf')  # <<< NEW: best max |error| on val
    best_state = copy.deepcopy(model.state_dict())
    best_ep    = -1

    start_time  = time.time()
    total_start = time.time()        # total wall-clock timer

    for ep in range(nEpoch):

        # ----- Stop if we've done lbfgs_epochs of L-BFGS -----
        if used_lbfgs and lbfgs_start_ep is not None:
            if ep - lbfgs_start_ep >= lbfgs_epochs:
                print(f"Stopping after {lbfgs_epochs} LBFGS epochs at global epoch {ep}")
                break

        # ------------------------------
        # STAGE 1: Adam (ep < switch_epoch)
        # STAGE 2: L-BFGS (ep >= switch_epoch)
        # ------------------------------
        if ep < switch_epoch:
            # ----- Adam update -----
            optimizer.zero_grad()
            
            # Compute predictions for training data (u)
            u_pred = net_u(x_tf, t_tf, k_tf, model)
            # Compute PDE residual at collocation points
            u_f_pred = net_f(x_f_tf, t_f_tf, k_f_tf, model)
    
            loss_PDE  = criterion(u_f_pred, torch.zeros_like(u_f_pred))
            loss_data = criterion(u_tf, u_pred)
            loss = loss_PDE + 100 * loss_data
    
            loss.backward()
            optimizer.step()
            
        else:
            # ----- Switch to L-BFGS once -----
            if not used_lbfgs:
                # 1) Load the best Adam weights BEFORE creating LBFGS
                model.load_state_dict(best_state)
        
                # 2) Print which Adam state you're starting from
                print(
                    f"Switching to L-BFGS at epoch {ep} "
                    f"-> starting from Adam best at epoch {best_ep} "
                    f"(TrainLoss={best_TL:.3e}, Val MSE={best_v:.3e})"
                )
        
                # 3) Create the LBFGS optimiser on top of that state
                optimizer = torch.optim.LBFGS(
                    model.parameters(),
                    max_iter=20,          # internal LBFGS iterations per .step()
                    history_size=100,
                    line_search_fn=None
                )
                used_lbfgs = True
                lbfgs_start_ep = ep      # <-- remember when we switched
                

            # L-BFGS requires a closure that re-computes the loss
            def closure():
                optimizer.zero_grad()
                u_pred   = net_u(x_tf, t_tf, k_tf, model)          # <<< CHANGED
                u_f_pred = net_f(x_f_tf, t_f_tf, k_f_tf, model)    # <<< CHANGED

                loss_PDE  = criterion(u_f_pred, torch.zeros_like(u_f_pred))
                loss_data = criterion(u_tf, u_pred)
                loss      = loss_PDE + 100 * loss_data

                loss.backward()
                return loss

            loss = optimizer.step(closure)  # returns the loss from the last closure call

        
        # ----- validation -----
        model.eval()
        with torch.no_grad():
            u_v_pred = net_u(x_v_tf, t_v_tf, k_v_tf, model)   # <<< CHANGED
            mse_v = criterion(u_v_pred, u_true_tf).item()
            mse_v_hist.append((ep, mse_v))

            # <<< NEW: max absolute error on validation set >>>
            abs_err = torch.abs(u_v_pred - u_true_tf)   # (N_val, 1)
            max_err = abs_err.max().item()              # scalar
            max_errors.append((ep, max_err))            # store (epoch, max_err)
            # -----------------------------------------------

        model.train()  # switch back


        # ----- early stopping on val -----
        # if mse_v < best_v:
        if loss.item() < best_TL:
            best_v       = mse_v
            best_TL      = loss.item()
            best_max_err = max_err          # <<< NEW: store max error at best state
            best_state   = copy.deepcopy(model.state_dict())
            best_ep      = ep
            # print(f"[Improved] Epoch {ep} | Best Val MSE: {best_v:.3e}")
            pat = 0
        else:
            pat += 1
            if pat >= patience:
                print(f"Early stopping at it={ep}, best Val MSE={best_v:.3e}")
                break
        
        # Print progress
        # - Before LBFGS: every 1000 epochs
        # - After LBFGS is enabled: every 100 epochs
        if (not used_lbfgs and ep % 1000 == 0) or (used_lbfgs and ep % 1000 == 0):
            elapsed = time.time() - start_time
            print(f"Epochs: {ep:6d} | TrainLoss: {loss.item():.3e} "
                  f"| Val MSE: {mse_v:.3e} "
                  f"| Max Val |err|: {max_err:.3e} "   # <<< NEW
                  f"| Time: {elapsed:.2f}s")
            start_time = time.time()
            
        loss_values.append(loss.item())

    total_elapsed = time.time() - total_start
    print(f"Total training time: {total_elapsed:.2f} s")
    print(f"Best Val MSE: {best_v:.3e} at epoch {best_ep}")
    print(f"Best Max |err| on validation: {best_max_err:.3e}")   # <<< NEW

    model.load_state_dict(best_state)          # <- load best here

    return loss_values, mse_v_hist, max_errors, best_ep, best_TL, best_v, best_max_err




# -----------------------------------------------------------------------------
# 1) Build dataset for arbitrary (N_i, N_b, N_k, N_f)
# -----------------------------------------------------------------------------
def build_dataset(N_i, N_b, N_k, N_f, N_val,
                  x_min, x_max,
                  t_min, t_max,
                  k_min, k_max,
                  seed):
    """
    Build training and collocation sets for the parametric heat equation.

    Returns:
        X_u_train : (N_u, 3) array of data points (x, t, k) for IC + BC.
        u_train   : (N_u, 1) IC + BC at X_u_train.
        X_f_train : (N_f, 3) collocation points in (x, t, k).
        X_val     : (N_val, 3) validation points in (x, t, k).
        lb, ub    : lower/upper bounds for normalisation in the NN.
    """

    
    # We sample k in [k_min, k_max].
    rng = np.random.default_rng(123)
    k_vec = rng.uniform(k_min, k_max, N_k)  # random in [k_min, k_max]


    # --- Initial condition: u(x,0;k) = sin(pi x) ---
    # x in [x_min, x_max], t=0, and all k samples
    x_ic = np.linspace(x_min, x_max, N_i)
    t_ic = np.array([t_min])   # or [t_min]
    x_ic_g, t_ic_g, k_ic_g = np.meshgrid(x_ic, t_ic, k_vec, indexing='ij')

    x_u_ic    = x_ic_g.ravel()[:, None]
    t_u_ic    = t_ic_g.ravel()[:, None]
    k_u_ic    = k_ic_g.ravel()[:, None]
    X_u_train_ic = np.hstack([x_u_ic, t_u_ic, k_u_ic])

    # --- Boundary conditions: u(0,t;k)=0, u(1,t;k)=0 ---
    # We discretise t with 2*N_b points between t_min and t_max.
    t_line = np.linspace(t_min, t_max, 2 * N_b)
    x_bc_left  = np.array([x_min])
    x_bc_right = np.array([x_max])
    x_bc = np.concatenate([x_bc_left, x_bc_right], axis=0)  # [0, 1]

    x_bc_g, t_bc_g, k_bc_g = np.meshgrid(x_bc, t_line, k_vec, indexing='ij')
    x_u_bc    = x_bc_g.ravel()[:, None]
    t_u_bc    = t_bc_g.ravel()[:, None]
    k_u_bc    = k_bc_g.ravel()[:, None]
    X_u_train_bc = np.hstack([x_u_bc, t_u_bc, k_u_bc])

    # --- Combine IC and BC into one "data" set ---
    X_u_train = np.vstack([X_u_train_ic, X_u_train_bc]).astype(np.float32)

    # --- Analytic solution for those IC/BC points ---
    x_cal    = X_u_train[:, 0]
    t_cal    = X_u_train[:, 1]
    k_cal    = X_u_train[:, 2]

    # Closed-form solution: u(x,t;k) = exp(-k π² t) sin(π x)
    u_train = np.exp(-k_cal * (np.pi**2) * t_cal) * np.sin(np.pi * x_cal)
    u_train = u_train[:, None].astype(np.float32)

    # --- Collocation + validation via LHS in (x, t, k) ---
    lb = np.array([x_min, t_min, k_min], dtype=np.float32)
    ub = np.array([x_max, t_max, k_max], dtype=np.float32)

    np.random.seed(seed)
    U_all = lhs(3, samples=N_f + N_val)   # Latin Hypercube in [0,1]^3
    X_all = lb + (ub - lb) * U_all        # map to [lb, ub] in (x, t, k)
    X_f_train = X_all[:N_f]
    X_val     = X_all[N_f:]

    return X_u_train, u_train, X_f_train, X_val, lb, ub


# -----------------------------------------------------------------------------
# 2) Compute global relative L2 error for a trained model at a fixed k
# -----------------------------------------------------------------------------
def compute_rel_L2(model,
                   x_min, x_max,
                   t_min, t_max,
                   k_val=1.0,
                   Nx=100, Nt=100,
                   device=device):
    """
    Compute global relative L2 error of the model solution against the analytic
    solution on a regular (x,t) grid at a fixed physical k = k_val.

    rel_L2 = ||u_pred - u_true||_2 / ||u_true||_2
    """

    # Build regular grid in x, t
    x_test = np.linspace(x_min, x_max, Nx)
    t_test = np.linspace(t_min, t_max, Nt)

    T, X = np.meshgrid(t_test, x_test, indexing='ij')  # shape (Nt, Nx)
    K    = np.full_like(T, k_val)                   # broadcast

    # Flatten to (Nt*Nx, 1) column vectors
    x_flat    = X.ravel()[:, None]
    t_flat    = T.ravel()[:, None]
    k_flat    = K.ravel()[:, None]

    # Stack into (Nt*Nx, 3) array and convert to torch tensor
    X_star    = np.hstack([x_flat, t_flat, k_flat]).astype(np.float32)
    X_star_tf = torch.from_numpy(X_star).to(device)

    # NN prediction over the grid
    model.eval()
    with torch.no_grad():
        u_pred = model(X_star_tf).squeeze(1).cpu().numpy().reshape(T.shape)

    # Analytic solution on the same grid
    u_true = u_true_numpy(X, T, K)

    # Global relative L2 error
    num = np.linalg.norm(u_pred - u_true)
    den = np.linalg.norm(u_true)
    rel_L2 = num / den if den > 0 else num

    return rel_L2


# -----------------------------------------------------------------------------
# 3) Plot training curves (loss, val MSE, max |error|) and save to file
# -----------------------------------------------------------------------------
def plot_training_curves(loss_values, mse_v_hist, max_errors,
                         N_i, N_b, N_k, N_f,
                         out_dir="sweep_results"):
    """
    Make the TrainLoss / ValMSE / Max|Error| vs epoch plot and save to file.

    Args:
        loss_values : list of train loss per epoch
        mse_v_hist  : list of (epoch, val_MSE)
        max_errors  : list of (epoch, max_abs_error) on validation
        N_i, N_b, N_k, N_f : configuration used (for filename)
        out_dir     : directory where the PNG is saved
    """

    os.makedirs(out_dir, exist_ok=True)

    # Training loss epochs
    ep_train = range(len(loss_values))

    # Validation MSE: unpack (epoch, mse)
    ep_val  = [int(i) for i, _ in mse_v_hist]
    mse_val = [
        (m.detach().cpu().item() if torch.is_tensor(m) else float(m))
        for _, m in mse_v_hist
    ]

    # Max absolute error: unpack (epoch, max_err)
    ep_max   = [int(i) for i, _ in max_errors]
    max_errs = [
        (m.detach().cpu().item() if torch.is_tensor(m) else float(m))
        for _, m in max_errors
    ]

    # Plot curves on log scale (since errors/loss typically span many orders)
    plt.figure(figsize=(8, 6))
    plt.plot(ep_train, loss_values, color='black', label='Train Loss')
    plt.plot(ep_val,   mse_val,     color='red',   label='Validation MSE')
    plt.plot(ep_max,   max_errs,    color='blue',  label='Max |Error| (Validation)')

    plt.xlabel('Iteration')
    plt.ylabel('Loss / Error')
    plt.yscale('log')
    plt.title('Training Loss, Validation MSE, and Max Validation Error vs Iterations')
    plt.legend()
    plt.tight_layout()

    # File name tagged with the configuration
    fname = f"train_Ni{N_i}_Nb{N_b}_Nk{N_k}_Nf{N_f}.png"
    fpath = os.path.join(out_dir, fname)
    plt.savefig(fpath, dpi=300, bbox_inches='tight')
    plt.close()

    return fpath


# -----------------------------------------------------------------------------
# 4) Run a single experiment for given (N_i, N_b, N_k, N_f)
# -----------------------------------------------------------------------------
def run_single_experiment(N_i, N_b, N_k, N_f, seed,
                          N_val=100,
                          Train_epochs=100000,
                          learning_rate=5e-4,
                          k_val_eval=1.0,
                          results_dir="sweep_results"):
    """
    Runs one full experiment:
      - builds dataset for given (N_i, N_b, N_k, N_f)
      - trains a fresh model
      - saves training curve plot
      - computes global rel L2 error at k = k_val_eval
      - returns a dict with all requested info
    """

    # 1) Build dataset (IC+BC data, collocation, validation, bounds)
    X_u_train, u_train, X_f_train, X_val, lb, ub = build_dataset(
        N_i=N_i, N_b=N_b, N_k=N_k, N_f=N_f, N_val=N_val,
        x_min=x_min, x_max=x_max, t_min=t_min, t_max=t_max,
        k_min=k_min, k_max=k_max, seed=seed
    )

    # 2) Initialise a new PINN model for this dataset
    model = NeuralNet(layers, lb, ub).to(device).float()

    # 3) Train and measure total wall-clock time for this configuration
    exp_start = time.time()
    loss_values, mse_v_hist, max_errors, best_ep, best_TL, best_v, best_max_err = train(
        Train_epochs,
        X_u_train,
        u_train,
        X_f_train,
        X_val,
        model,
        learning_rate
    )
    total_elapsed = time.time() - exp_start

    # 4) Compute global relative L2 error at a chosen k (e.g. k = 1.0)
    rel_L2 = compute_rel_L2(
        model,
        x_min=x_min, x_max=x_max,
        t_min=t_min, t_max=t_max,
        k_val=k_val_eval,
        Nx=100, Nt=100,
        device=device
    )

    # 5) Create and save the training curve plot for this run
    curve_path = plot_training_curves(
        loss_values, mse_v_hist, max_errors,
        N_i=N_i, N_b=N_b, N_k=N_k, N_f=N_f,
        out_dir=results_dir
    )

    # 6) Pack all information into a record dictionary
    record = {
        "N_i": N_i,
        "N_b": N_b,
        "N_k": N_k,
        "N_f": N_f,
        "total_elapsed": total_elapsed,
        "best_ep": best_ep,
        "best_TL": best_TL,
        "best_v": best_v,
        "best_max_err": best_max_err,
        "rel_L2": rel_L2,
        "loss_values": loss_values,
        "mse_v_hist": mse_v_hist,
        "max_errors": max_errors,
        "training_curve_path": curve_path,
    }

    return record


# -----------------------------------------------------------------------------
# 5) Sweep settings and loops for N_f, N_i, N_b, N_k
# -----------------------------------------------------------------------------
# Network architecture: [ (x, t, k) ] -> hidden layers -> [u]
layers = [3, 50, 50, 50, 1]

# Domain bounds
x_min, x_max = 0.0, 1.0
t_min, t_max = 0.0, 0.25
k_min, k_max = -5.0, 5.0

# Random seed for reproducibility
seed = 123
    
# Base values (same as your current defaults)
BASE_N_i = 101
BASE_N_b = 51
BASE_N_k = 51
BASE_N_f = 1000

# Training hyperparameters for all sweeps
Train_epochs = 100000
learning_rate = 0.0005
N_val = 100          # number of validation points from LHS
k_val_eval = 1.0     # k at which global rel L2 is computed



# ==============================
# Repeat experiment: fixed N_f
# ==============================
fixed_N_f   = None      # set to an int (e.g. 1000) to override, or None to use BASE_N_f
N_repeats   = 5         # how many times to repeat
results_rep = []

for rep in range(1, N_repeats + 1):
    # Decide which N_f to use
    N_f_to_use = fixed_N_f if fixed_N_f is not None else BASE_N_f
    
    print(
        f"\n=== Repeat {rep}/{N_repeats} with "
        f"N_f = {N_f_to_use} (N_i={BASE_N_i}, N_b={BASE_N_b}, N_k={BASE_N_k}) ==="
    )

    rec = run_single_experiment(
        N_i=BASE_N_i,
        N_b=BASE_N_b,
        N_k=BASE_N_k,
        N_f=N_f_to_use,  
        seed=123,                 # keep the same seed if you truly want identical setup
        # seed=123 + rep          # <- use this instead if you want different randomness per run
        N_val=N_val,
        Train_epochs=Train_epochs,
        learning_rate=learning_rate,
        k_val_eval=k_val_eval,
        results_dir="repeat",
    )
    results_rep.append(rec)

# Convert list of records to DataFrame for easy inspection and saving
df_rep = pd.DataFrame([
    {
        "repeat": i + 1,
        "N_i": r["N_i"], "N_b": r["N_b"], "N_k": r["N_k"], "N_f": r["N_f"],
        "total_elapsed": r["total_elapsed"],
        "best_ep": r["best_ep"],
        "best_TL": r["best_TL"],
        "best_v": r["best_v"],
        "best_max_err": r["best_max_err"],
        "rel_L2": r["rel_L2"],
    }
    for i, r in enumerate(results_rep)
])

print("\nRepeated runs summary:")
display(df_rep)

# Save repeated-runs summary table as Excel
df_rep.to_excel("repeat_summary.xlsx", index=False)



# # ==========================
# # Sweep 1: Collocation points N_f
# # ==========================
# Nf_list = [500, 1000, 2000, 4000]  # values to test for N_f

# results_Nf = []

# for N_f in Nf_list:
#     print(f"\n=== Sweep N_f = {N_f} (N_i={BASE_N_i}, N_b={BASE_N_b}, N_k={BASE_N_k}) ===")
#     rec = run_single_experiment(
#         N_i=BASE_N_i,
#         N_b=BASE_N_b,
#         N_k=BASE_N_k,
#         N_f=N_f,
#         seed=123,
#         N_val=N_val,
#         Train_epochs=Train_epochs,
#         learning_rate=learning_rate,
#         k_val_eval=k_val_eval,
#         results_dir="sweep_Nf",
#     )
#     results_Nf.append(rec)

# # Convert list of records to DataFrame for easy inspection and saving
# df_Nf = pd.DataFrame([
#     {
#         "N_i": r["N_i"], "N_b": r["N_b"], "N_k": r["N_k"], "N_f": r["N_f"],
#         "total_elapsed": r["total_elapsed"],
#         "best_ep": r["best_ep"],
#         "best_TL": r["best_TL"],
#         "best_v": r["best_v"],
#         "best_max_err": r["best_max_err"],
#         "rel_L2": r["rel_L2"],
#         # Histories stored as objects (lists) – still useful inside Python
#         # "loss_values": r["loss_values"],
#         # "mse_v_hist": r["mse_v_hist"],
#         # "max_errors": r["max_errors"],
#         # "training_curve_path": r["training_curve_path"],
#     }
#     for r in results_Nf
# ])

# print("\nN_f sweep summary:")
# display(df_Nf)

# # Save N_f sweep summary table as Excel
# df_Nf.to_excel("sweep_Nf_summary.xlsx", index=False)

# # ==========================
# # Sweep 2: IC points N_i
# # ==========================
# Ni_list = [21, 51, 101, 201]  # values to test for N_i

# results_Ni = []

# for N_i in Ni_list:
#     print(f"\n=== Sweep N_i = {N_i} (N_b={BASE_N_b}, N_k={BASE_N_k}, N_f={BASE_N_f}) ===")
#     rec = run_single_experiment(
#         N_i=N_i,
#         N_b=BASE_N_b,
#         N_k=BASE_N_k,
#         N_f=BASE_N_f,
#         N_val=N_val,
#         Train_epochs=Train_epochs,
#         learning_rate=learning_rate,
#         k_val_eval=k_val_eval,
#         results_dir="sweep_Ni",
#         seed=123
#     )
#     results_Ni.append(rec)

# df_Ni = pd.DataFrame([
#     {
#         "N_i": r["N_i"], "N_b": r["N_b"], "N_k": r["N_k"], "N_f": r["N_f"],
#         "total_elapsed": r["total_elapsed"],
#         "best_ep": r["best_ep"],
#         "best_TL": r["best_TL"],
#         "best_v": r["best_v"],
#         "best_max_err": r["best_max_err"],
#         "rel_L2": r["rel_L2"],
#         # "loss_values": r["loss_values"],
#         # "mse_v_hist": r["mse_v_hist"],
#         # "max_errors": r["max_errors"],
#         # "training_curve_path": r["training_curve_path"],
#     }
#     for r in results_Ni
# ])

# print("\nN_i sweep summary:")
# display(df_Ni)

# df_Ni.to_excel("sweep_Ni_summary.xlsx", index=False)

# # ==========================
# # Sweep 3: BC points N_b
# # ==========================
# Nb_list = [11, 21, 51, 101]  # values to test for N_b

# results_Nb = []

# for N_b in Nb_list:
#     print(f"\n=== Sweep N_b = {N_b} (N_i={BASE_N_i}, N_k={BASE_N_k}, N_f={BASE_N_f}) ===")
#     rec = run_single_experiment(
#         N_i=BASE_N_i,
#         N_b=N_b,
#         N_k=BASE_N_k,
#         N_f=BASE_N_f,
#         N_val=N_val,
#         Train_epochs=Train_epochs,
#         learning_rate=learning_rate,
#         k_val_eval=k_val_eval,
#         results_dir="sweep_Nb",
#         seed=123
#     )
#     results_Nb.append(rec)

# df_Nb = pd.DataFrame([
#     {
#         "N_i": r["N_i"], "N_b": r["N_b"], "N_k": r["N_k"], "N_f": r["N_f"],
#         "total_elapsed": r["total_elapsed"],
#         "best_ep": r["best_ep"],
#         "best_TL": r["best_TL"],
#         "best_v": r["best_v"],
#         "best_max_err": r["best_max_err"],
#         "rel_L2": r["rel_L2"],
#         # "loss_values": r["loss_values"],
#         # "mse_v_hist": r["mse_v_hist"],
#         # "max_errors": r["max_errors"],
#         # "training_curve_path": r["training_curve_path"],
#     }
#     for r in results_Nb
# ])

# print("\nN_b sweep summary:")
# display(df_Nb)

# df_Nb.to_excel("sweep_Nb_summary.xlsx", index=False)

# # ==========================
# # Sweep 4: parameter samples N_k
# # ==========================
# Nk_list = [11, 21, 51, 101]  # values to test for N_k

# results_Nk = []

# for N_k in Nk_list:
#     print(f"\n=== Sweep N_k = {N_k} (N_i={BASE_N_i}, N_b={BASE_N_b}, N_f={BASE_N_f}) ===")
#     rec = run_single_experiment(
#         N_i=BASE_N_i,
#         N_b=BASE_N_b,
#         N_k=N_k,
#         N_f=BASE_N_f,
#         N_val=N_val,
#         Train_epochs=Train_epochs,
#         learning_rate=learning_rate,
#         k_val_eval=k_val_eval,
#         results_dir="sweep_Nk",
#         seed=123
#     )
#     results_Nk.append(rec)

# df_Nk = pd.DataFrame([
#     {
#         "N_i": r["N_i"], "N_b": r["N_b"], "N_k": r["N_k"], "N_f": r["N_f"],
#         "total_elapsed": r["total_elapsed"],
#         "best_ep": r["best_ep"],
#         "best_TL": r["best_TL"],
#         "best_v": r["best_v"],
#         "best_max_err": r["best_max_err"],
#         "rel_L2": r["rel_L2"],
#         # "loss_values": r["loss_values"],
#         # "mse_v_hist": r["mse_v_hist"],
#         # "max_errors": r["max_errors"],
#         # "training_curve_path": r["training_curve_path"],
#     }
#     for r in results_Nk
# ])

# print("\nN_k sweep summary:")
# display(df_Nk)

# df_Nk.to_excel("sweep_Nk_summary.xlsx", index=False)

print("\n=== END OF SCRIPT ===")

CUDA available? True
Device: NVIDIA A2
Using device: cuda

=== Repeat 1/5 with N_f = 1000 (N_i=101, N_b=51, N_k=51) ===
Epochs:      0 | TrainLoss: 2.018e+01 | Val MSE: 5.257e+00 | Max Val |err|: 1.018e+01 | Time: 0.23s
Epochs:   1000 | TrainLoss: 3.442e+00 | Val MSE: 3.219e+00 | Max Val |err|: 8.426e+00 | Time: 3.41s
Epochs:   2000 | TrainLoss: 6.850e-01 | Val MSE: 4.797e-01 | Max Val |err|: 3.377e+00 | Time: 3.42s
Switching to L-BFGS at epoch 3000 -> starting from Adam best at epoch 2986 (TrainLoss=1.133e-01, Val MSE=3.646e-02)
Epochs:   3000 | TrainLoss: 1.133e-01 | Val MSE: 1.107e-02 | Max Val |err|: 4.899e-01 | Time: 3.39s
Epochs:   4000 | TrainLoss: 2.995e-05 | Val MSE: 3.598e-05 | Max Val |err|: 3.627e-02 | Time: 87.84s
Epochs:   5000 | TrainLoss: 2.995e-05 | Val MSE: 3.598e-05 | Max Val |err|: 3.627e-02 | Time: 8.67s
Stopping after 3000.0 LBFGS epochs at global epoch 6000
Total training time: 115.42 s
Best Val MSE: 3.598e-05 at epoch 3607
Best Max |err| on validation: 3.627e-02

,repeat,N_i,N_b,N_k,N_f,total_elapsed,best_ep,best_TL,best_v,best_max_err,rel_L2
0,1,101,51,51,1000,116.440816,3607,0.000030,0.000036,0.036273,0.000835
1,2,101,51,51,1000,143.462456,3895,0.000034,0.000015,0.024312,0.000670
2,3,101,51,51,1000,135.137108,3868,0.000026,0.000050,0.051735,0.001172
3,4,101,51,51,1000,166.219461,4341,0.000025,0.000049,0.038024,0.001209
4,5,101,51,51,1000,114.364171,3656,0.000022,0.000013,0.023948,0.000545



=== END OF SCRIPT ===


In [2]:
# t = data['t'].flatten()[:,None] # read in t and flatten into column vector
# x = data['x'].flatten()[:,None] # read in x and flatten into column vector
#  # Exact represents the exact solution to the problem, from the data provided
# Exact = np.real(data['usol']).T # Exact has structure of nt times nx

# print("usol shape (nt, nx) = ", Exact.shape)

# # We need to find all the x,t coordinate pairs in the domain
# X, T = np.meshgrid(x,t)

# # Flatten the coordinate grid into pairs of x,t coordinates
# X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None])) # coordinates x,t
# u_star = Exact.flatten()[:,None]   # corresponding solution value with each coordinate


# print("X has shape ", X.shape, ", X_star has shape ", X_star.shape, ", u_star has shape ", u_star.shape)

# # Domain bounds (-1,1)
# lb = X_star.min(axis=0)
# ub = X_star.max(axis=0)

# print("Lower bounds of x,t: ", lb)
# print("Upper bounds of x,t: ", ub)
# print('')
# print('The first few entries of X_star are:')
# print( X_star[0:5, :] )

# print('')
# print('The first few entries of u_star are:')
# print( u_star[0:5, :] )